In [1]:
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

In [2]:
import os
import pandas as pd
import numpy as np
import math

os.chdir("../")
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F

### Constants ###
SAMPLING_RATE = 2
CAMERA_ID = "scene-0757"

CAMERA_COLUMNS = [
    "cameraId",
    "frameId",
    "frameNum",
    "filename",
    "cameraTranslation",
    "cameraRotation",
    "cameraIntrinsic",
    "egoTranslation",
    "egoRotation",
    "timestamp",
    "cameraHeading",
    "egoHeading",
    "cameraTranslationAbs"] #road_direction not included yet

def convert_frame_to_map(frame):
    map_frame = dict(zip(CAMERA_COLUMNS, frames[:12]))
    return map_frame

def transform_to_world(frame_coordinate, ego_translation, ego_rotation):
    ### TODO: get world coordinates
    return frame_coordinate

def get_obj_trajectory(tracking_df, ego_config):
    '''
    returned object info is a dictionary that looks like this:
    {object_id:{frame_idx:[], #need to use the frame idx of the video to get the camera config for each frame
                trajectory:[]}
    '''
    obj_info = {}
    grouped_trajectory = tracking_df.groupby(by=["object_id"])
    for name, group in grouped_trajectory:
        obj_info[name] = {}
        group.frame_idx = group.frame_idx.add(-1)
        object_df = group[[
            'frame_idx', 'object_id', 'object_type', 'bbox_left', 'bbox_top', 'bbox_w', 'bbox_h']]
        object_df = object_df.reset_index(drop=True)
        framenums = group.frame_idx.tolist()
        
        ### get ego_config for each framenum
        transformation_config = ego_config.iloc[framenums]
        transformation_config = transformation_config.reset_index(drop=True)
        
        object_with_ego = pd.concat([object_df, transformation_config], axis=1)
        ### for each coordinate, transform
        obj_trajectory = []
        for index, row in object_with_ego.iterrows():
            obj_trajectory.append(transform_to_world(frame_coordinate=((row['bbox_left'], row['bbox_top']), 
                                                 (row['bbox_left']+row['bbox_w'], 
                                                  row['bbox_top']+row['bbox_h'])), 
                               ego_translation=row['egoTranslation'],
                               ego_rotation=row['egoRotation']))
        
        obj_info[name]['frame_idx'] = object_with_ego[['frame_idx']]
        obj_info[name]['trajectory'] = obj_trajectory
    return obj_info


def facing_relative(prev_traj_point, next_traj_point, current_ego_heading):
    ### TODO: get direction from adjacent traj points, then calculate the relative degree
    ####### COMPLETE
    diff = next_traj_point - prev_traj_point
    diff_heading = math.degrees(np.arctan2(diff[1], diff[0])) - 90
    result = ((diff_heading - current_ego_heading) % 360 + 360) % 360
    return result

def facing_relative_check(obj_info, threshold):
    for obj_id in obj_info:
        frame_idx = obj_info[obj_id]['frame_idx'].frame_idx
        trajectory = obj_info[obj_id]['trajectory']
        ego_heading = ego_config.iloc[frame_idx.tolist()].egoHeading.tolist()
        print(frame_idx[:len(ego_heading)-1][[facing_relative(trajectory[i], trajectory[i+1], ego_heading[i]) > threshold for i in range(len(ego_heading)-1)]])

"""
Dummy Modules for playing around
"""
class optimizeRoadNetwork:
    def __init__(self, road_network=None):
        self.road_network = road_network
        self.optimized_road_network = self.optimize_road_network()

    def optimize_road_network(self):
        return self.road_network

    def optimize_filter_intersection(self, sample_frames):
        intersection_filtered = []

        # TODO: Connection to DB for each execution might take too much time, do all at same time
        for frame in sampled_frames:
            map_frame = convert_frame_to_map(frame)
            # use sql in order to make use of mobilitydb features. TODO: Find python alternative
            query = f"SELECT TRUE WHERE minDistance('{map_frame['egoTranslation']}', 'intersection') < 10" 
            result = database._execute_query(query)
            if result:
                intersection_filtered.append(frame)

        return intersection_filtered


class optimizeSampling:
    def __init__(self, sampling=None):
        self.sampling = sampling
        self.optimized_sampling = self.optimize_sampling()

    def optimize_sampling(self):
        return self.sampling
    
    def native_sample(self):
        # Sample All Frames from Video at a #
        query = f"SELECT * FROM Cameras WHERE cameraId = '{CAMERA_ID}' ORDER BY frameNum"

        all_frames = database._execute_query(query)
        sampled_frames = all_frames[::SAMPLING_RATE]
        return sampled_frames


class optimizeDecoding:
    def __init__(self, decoding=None):
        self.decoding = decoding
        self.optimized_decoding = self.optimize_decoding()

    def optimize_decoding(self):
        return self.decoding

    def optimize_tracking(self, lst_of_frames):
        # Now the result is written to a txt file, need to fix this later
        from Yolov5_Strong_Detection import sample_frame_tracker
        for frames in lst_of_frames:
            result = sample_frame_tracker.run(frames, save_vid=True)

"""
End to End Optimization Ingestion
Use Case: select cars appears in intersection
                      facing degree d relative to ego
Test Data: only use one video as the test data
"""


class optimizeIngestion:
    def __init__(self):
        self.optimize_road_network = optimizeRoadNetwork()
        self.optimize_sampling = optimizeSampling()
        self.optimize_decoding = optimizeDecoding()

    def run_test(self):
        # 1. Get all frames from video
        all_frames = self.optimize_sampling.native_sample()
        
        
        # 2. Filter out frames that in intersection
        intersection_filtered = self.optimize_road_network.optimize_filter_intersection(all_frames)
        ###TODO:fetch the camera_config corresponding to intersection_filtered 
        query = """SELECT * FROM Cameras 
                    WHERE filename like 'samples/CAM_FRONT/%2018-08-01-15%' 
                    ORDER BY frameNum""" 
        camera_config = database._execute_query(query)
        camera_config_df = pd.DataFrame(camera_config, columns=camera_columns)
        ego_config = camera_config_df[['egoTranslation', 'egoRotation', 'egoHeading']]
        
        
        # 3. Decode filtered_frames and track
        self.optimize_decoding.optimize_tracking([intersection_filtered])
        df = pd.read_csv("../optimization_playground/tracks/CAM_FRONT.txt", sep=" ", header=None, 
                 names=["frame_idx", 
                        "object_id", 
                        "bbox_left", 
                        "bbox_top", 
                        "bbox_w", 
                        "bbox_h", 
                        "None1",
                        "None2",
                        "None3",
                        "None",
                        "object_type"])
        
        obj_info = get_obj_trajectory(df)
        facing_relative_check(obj_info, 0)

'\nWhat do we have?\nNow everything still in the mobilitydb\n\n1. Test on mini, CAM_FRONT images treated as raw videos\n2. camera config for all the CAM_FRONT ego, each row is a single camera also a single image\n3. road network ingested in the way for scenic case, may need to dig out other useful stuff\n'

In [3]:
%cd ../Yolov5_StrongSORT_OSNet

/home/yongming/workspace/research/apperception_new_local/apperception/Yolov5_StrongSORT_OSNet


In [4]:
import glob
files = sorted(glob.glob('/home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/*'))

In [5]:
test_frames = [f for f in files if '2018-08-01-15' in f]
len(test_frames)

40

In [6]:
import sample_frame_tracker
def track_sample(lst_of_frames):
    for frames in lst_of_frames:
        result = sample_frame_tracker.run(frames, save_vid=True)

strong_sort/deep/reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
/home/yongming/.pyenv/versions/3.8.3/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
YOLOv5 🚀 2022-8-20 Python-3.8.3 torch-1.12.1+cu102 CPU

Fusing layers... 


strong_sort/configs/strong_sort.yaml


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


In [7]:
track_sample([test_frames])

Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


image 1/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151603512404.jpg: 384x640 9 persons, 6 cars, 4 traffic lights, Done. YOLO:(0.253s), StrongSORT:(0.569s)
image 2/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604012404.jpg: 384x640 9 persons, 7 cars, 3 traffic lights, 1 fire hydrant, Done. YOLO:(0.254s), StrongSORT:(0.549s)
image 3/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604512404.jpg: 384x640 10 persons, 8 cars, 5 traffic lights, Done. YOLO:(0.245s), StrongSORT:(0.796s)
image 4/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151605012404.jpg: 384x640 8 persons, 11 cars, 6 traffic lights, 1 fire hydrant, Done. YOLO:(0.321s), StrongSORT:(0.717s)
image 5/40 /home/yongm

image 36/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151621012404.jpg: 384x640 18 cars, 1 truck, Done. YOLO:(0.229s), StrongSORT:(0.555s)
image 37/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151621412404.jpg: 384x640 20 cars, 1 truck, Done. YOLO:(0.239s), StrongSORT:(0.564s)
image 38/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151621912404.jpg: 384x640 19 cars, Done. YOLO:(0.288s), StrongSORT:(0.704s)
image 39/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151622412404.jpg: 384x640 1 bicycle, 18 cars, 1 umbrella, Done. YOLO:(0.212s), StrongSORT:(0.529s)
image 40/40 /home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_F

In [1]:
import pandas as pd
df = pd.read_csv("../optimization_playground/tracks/CAM_FRONT.txt", sep=" ", header=None, 
                 names=["frame_idx", 
                        "object_id", 
                        "bbox_left", 
                        "bbox_top", 
                        "bbox_w", 
                        "bbox_h", 
                        "None1",
                        "None2",
                        "None3",
                        "None",
                        "object_type"])

In [2]:
df

,frame_idx,object_id,bbox_left,bbox_top,bbox_w,bbox_h,None1,None2,None3,None,object_type
0,3,1,996,485,61,49,-1,-1,-1,0,car
1,3,4,1400,476,32,103,-1,-1,-1,0,person
2,3,10,1083,470,41,40,-1,-1,-1,0,car
3,4,1,976,463,67,50,-1,-1,-1,0,car
4,4,4,1360,461,34,99,-1,-1,-1,0,person
...,...,...,...,...,...,...,...,...,...,...,...
518,40,210,853,481,53,43,-1,-1,-1,0,car
519,40,212,1564,525,35,188,-1,-1,-1,0,car
520,40,233,264,484,153,120,-1,-1,-1,0,car
521,40,239,932,479,37,38,-1,-1,-1,0,car


In [3]:
%cd ..

/home/yongming/workspace/research/apperception_new_local/apperception


In [4]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F

In [5]:
camera_columns = [
    "cameraId",
    "frameId",
    "frameNum",
    "filename",
    "cameraTranslation",
    "cameraRotation",
    "cameraIntrinsic",
    "egoTranslation",
    "egoRotation",
    "timestamp",
    "cameraHeading",
    "egoHeading",
    "cameraTranslationAbs"]

In [6]:
# Sample All Frames from Video at a #
query = f"SELECT * FROM Cameras WHERE filename like 'samples/CAM_FRONT/%2018-08-01-15%' ORDER BY frameNum"

In [7]:
camera_config = database._execute_query(query)
camera_config_df = pd.DataFrame(camera_config, columns=camera_columns)
ego_config = camera_config_df[['egoTranslation', 'egoRotation', 'egoHeading']]

In [11]:
def transform_to_world(frame_coordinate, ego_translation, ego_rotation):
    ### TODO: get world coordinates
    return frame_coordinate

In [65]:
def get_obj_trajectory(tracking_df):
    '''
    returned object info is a dictionary that looks like this:
    {object_id:{frame_idx:[], #need to use the frame idx of the video to get the camera config for each frame
                trajectory:[]}
    '''
    obj_info = {}
    grouped_trajectory = tracking_df.groupby(by=["object_id"])
    for name, group in grouped_trajectory:
        obj_info[name] = {}
        group.frame_idx = group.frame_idx.add(-1)
        object_df = group[[
            'frame_idx', 'object_id', 'object_type', 'bbox_left', 'bbox_top', 'bbox_w', 'bbox_h']]
        object_df = object_df.reset_index(drop=True)
        framenums = group.frame_idx.tolist()
        
        ### get ego_config for each framenum
        transformation_config = ego_config.iloc[framenums]
        transformation_config = transformation_config.reset_index(drop=True)
        
        object_with_ego = pd.concat([object_df, transformation_config], axis=1)
        ### for each coordinate, transform
        obj_trajectory = []
        for index, row in object_with_ego.iterrows():
            obj_trajectory.append(transform_to_world(frame_coordinate=((row['bbox_left'], row['bbox_top']), 
                                                 (row['bbox_left']+row['bbox_w'], 
                                                  row['bbox_top']+row['bbox_h'])), 
                               ego_translation=row['egoTranslation'],
                               ego_rotation=row['egoRotation']))
        
        obj_info[name]['frame_idx'] = object_with_ego[['frame_idx']]
        obj_info[name]['trajectory'] = obj_trajectory
    return obj_info

In [66]:
obj_info = get_obj_trajectory(df)

In [67]:
def facing_relative(prev_traj_point, next_traj_point, current_ego_heading):
    ### TODO: get direction from adjacent traj points, then calculate the relative degree
    return 1

In [68]:
def facing_relative_check(obj_info, threshold):
    for obj_id in obj_info:
        frame_idx = obj_info[obj_id]['frame_idx'].frame_idx
        trajectory = obj_info[obj_id]['trajectory']
        ego_heading = ego_config.iloc[frame_idx.tolist()].egoHeading.tolist()
        print(frame_idx[:len(ego_heading)-1][[facing_relative(trajectory[i], trajectory[i+1], ego_heading[i]) > threshold for i in range(len(ego_heading)-1)]])
        

In [69]:
facing_relative_check(obj_info, 0)

0      2
1      3
2      4
3      5
4      6
5      7
6      8
7      9
8     10
9     11
10    12
11    13
12    14
13    15
14    16
15    17
16    18
17    19
18    20
19    23
20    24
21    25
22    26
23    27
24    28
25    29
26    30
27    31
28    32
29    33
30    34
Name: frame_idx, dtype: int64
0      2
1      3
2      4
3      5
4      6
5      8
6      9
7     10
8     11
9     12
10    13
11    14
12    15
13    16
14    18
15    19
16    20
17    21
18    22
Name: frame_idx, dtype: int64
0      2
1      3
2      4
3      5
4      6
5      7
6      8
7      9
8     10
9     11
10    12
11    13
12    14
13    15
14    16
15    17
16    18
17    19
18    20
19    21
20    29
21    30
22    34
23    35
24    36
25    37
26    38
Name: frame_idx, dtype: int64
0     3
1     4
2     5
3     7
4     8
5     9
6    10
7    11
8    12
Name: frame_idx, dtype: int64
0      6
1      7
2      8
3      9
4     10
5     11
6     12
7     14
8     15
9     16
10    17
11    18
12    2